In [1]:
import requests
from time import sleep
from pprint import pprint
import json
from tqdm import tqdm

In [30]:
# VK API
class VkGet:
    url = 'https://api.vk.com/method/'
    def __init__(self, vk_token, version = '5.131'):
         self.params = {
            'access_token': vk_token,
            'v': version
            }

    def get_photos(self, owner_id = None, album_id = 'profile', count = 5):
        """Метод получает фотографии с профиля, по умолчанию - аватарки владельца профиля в количестве 5 штук.
            Возращает массив в формате json """
        get_photos_params = {
            'owner_id': owner_id,
            'album_id':  album_id,
            'rev': 1,
            'extended': 1,
            'count': count
            }
        get_photos_url = self.url + 'photos.get'
        response = requests.get(get_photos_url, params={**self.params, **get_photos_params}).json()
        return response['response']['items']

    def json_photos(self, lst):
        """Метод формирует json файл с информацией о фотографиях"""
        json_result = []
        for i, photo in enumerate(lst):
            json_result.append({
                # поскольку есть файлы с одинаковым кол-вом лайков, в название добавлен порядковый номер
                'file_name': '_'.join([str(i + 1), str(photo['likes']['count'])]),
                'size': photo['sizes'][-1]['type']
            })
        jsonString = json.dumps(json_result)
        with open("ben_photos.json", "w") as jsonFile:
            jsonFile.write(jsonString)


# Yandex API
class YaUploader:
    def __init__(self, token: str):
        self.ya_token = token

    def get_headers(self):
        return {'Content-type': 'application/json',
            'Authorization': f'OAuth {self.ya_token}'}

    def create_dir(self):
        """Метод создает папку на Яндекс-диске"""
        url = 'https://cloud-api.yandex.net/v1/disk/resources'
        headers = self.get_headers()
        params = {'path': 'Vk_photo'}
        response = requests.put(url, headers = headers, params = params)

    def upload(self, lst):
        """Метод загружает фотографии на яндекс диск и формирует json c информацией о них"""
        pbar = tqdm(total = len(lst))
        url = 'https://cloud-api.yandex.net/v1/disk/resources/upload'
        headers = self.get_headers()
        for i, photo in enumerate(lst):
            pbar.update(1)
            file_name = '_'.join([str(i + 1), str(photo['likes']['count'])])
            path = f'Vk_photo/{file_name}.jpeg'
            params = {'path': path, 'url': photo['sizes'][-1]['url']}
            response = requests.post(url, headers = headers, params = params)
        pbar.close()

In [31]:
if __name__ == '__main__':
    vk_token = input("Введите токен vk: ")
    vk_result = VkGet(vk_token)
    vk_result.json_photos(vk_result.get_photos())
    
    ya_token = input("Введите токен Яндекса: ")
    uploader = YaUploader(ya_token)
    
    uploader.create_dir()
    uploader.upload(vk_result.get_photos())

100%|██████████| 5/5 [00:02<00:00,  1.96it/s]
